# 예제: Tensorboard를 이용한 학습과정 시각화


---
이 실습파일은 선형회귀 실습 예제 파일을 수정한 것입니다. tf.summary API를 추가한 것 외에는 동일합니다.

Tensorflow 라이브러리를 import합니다.

**Note-1:** 2025.03.18 PC로 log file을 다운로드해서 tensorboard를 실행하는 방식에서 colab 자체에서 tensorboard를 실행하는 방식으로 변환함.

**Note-2:** 2025.03.18 tf version=2.18.0

In [ ]:
import tensorflow as tf
tf.__version__

weight W, bias B 변수를 선언하고, 선형회귀 모델을 정의합니다.

In [2]:
# 선형회귀 모델(Wx + b)과 tf.Variable을 선언합니다.
W = tf.Variable(tf.random.normal(shape=[1]))
b = tf.Variable(tf.random.normal(shape=[1]))

@tf.function
def linear_model(x):
  return W*x + b


예측값과 목적값과의 차이를 나타내는 손실함수를 평균제곱오차로 정의합니다.

In [3]:
# 손실 함수를 정의합니다.
# MSE 손실함수 \mean{(y' - y)^2}
@tf.function
def mse_loss(y_pred, y):
  return tf.reduce_mean(tf.square(y_pred - y))


최적화 함수로 그라디언트 디센트 옵티마이저를 정의합니다.

In [4]:
# 최적화를 위한 그라디언트 디센트 옵티마이저를 정의합니다.
optimizer = tf.optimizers.SGD(0.01)


여기서부터는 앞에서 실습한 코드와 조금 달라지는데,

1.   먼저 colab의 tensorboard extension을 불러오고,
2.   tf.summary 라이브러리의 create_file_writer함수로 스텝별 손실함수 값을 저장할 폴더 경로를 지정합니다.

In [5]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# 텐서보드 summary 정보들을 저장할 폴더 경로를 설정합니다.
summary_writer = tf.summary.create_file_writer('./tensorboard_log')


여기에서 W와 b값의 최적화 업데이트를 위한 함수를 정의하는데, 이때 이 함수가 한번 실행될 때마다, tf.summary 라이브러리의 scalar함수를 사용해서 loss 손실함수 값이 기록되도록 코드에 추가했습니다.
그 아랫부분은 앞서 실습했던 선형회귀 부분과 동일하게, W와 b에 대해 loss값을 미분하고, 이 미분값을 적용해서 W와 b값을 업데이트 하게 됩니다.


In [6]:
# 최적화를 위한 학습과 학습결과를 Tensorboar에 기록할 함수를 정의합니다.
@tf.function
def train_step(x, y):
  with tf.GradientTape() as tape:
    y_pred = linear_model(x)
    loss = mse_loss(y_pred, y)
  with summary_writer.as_default():
    tf.summary.scalar('loss', loss, step=optimizer.iterations)
  gradients = tape.gradient(loss, [W, b])
  optimizer.apply_gradients(zip(gradients, [W, b]))


이 아래 부분은 선형회귀와 동일하게, 트레이닝 데이터를 준비하고, 경사하강법을 1000번 실행한 후에, 학습이 잘 되었는지 테스트 데이터를 넣어서 정확도를 측정해보는 것입니다.

In [7]:
# 트레이닝을 위한 입력값과 출력값을 준비합니다.
x_train = [1, 2, 3, 4]
y_train = [2, 4, 6, 8]


In [8]:
# 경사하강법을 1000번 수행합니다.
# 1000번 수행마다, 'tensorboard_log' 폴더에 loss값이 log파일에 기록됩니다.
for i in range(1000):
  train_step(x_train, y_train)


In [9]:
# 테스트를 위한 입력값을 준비합니다.
x_test = [3.5, 5, 5.5, 6]
# 테스트 데이터를 이용해 학습된 선형회귀 모델이 데이터의 경향성(y=2x)을 잘 학습했는지 측정합니다.
# 예상되는 참값 : [7, 10, 11, 12]
print(linear_model(x_test).numpy())


[ 7.0024896 10.009158  11.011381  12.013604 ]


**colab tensorboard extenstion을 실행해서, 학습과정을 시각화합니다.**

In [ ]:
%tensorboard --logdir ./tensorboard_log

**tensorboard를 실행해서, 학습과정을 시각화하기 위해서는 아래의 과정을 따라합니다.(2025.03.18시점 아래 방법은 동작하지 않음-PC에 설치된 tensorflow version이 log file format을 제대로 읽지 못함)**

1.   'tensorboard_log' 폴더에 있는 log file을 본인의 PC에 다운로드 받습니다.(코랩에서 왼쪽에 보이는 파일 폴더를 누르면, 현재 디렉터리의 폴더들과 파일들이 보이는데, 여기에서 tensorboard_log 폴더를 눌러보면 events로 시작되는 로그 파일이 보입니다.
이 로그파일을 더블클릭해서 보면 알겠지만, 학습을 진행하면서 loss값이 계속 누적되어서 저장되어 있는 것이 보입니다.)
2.   windows의 "명령 프롬프트"나 "powershell"앱을 log file이 위치한 곳에서 띄워서, '$tensorboard --logdir=./' 명령을 실행합니다.
3. Chrome이나 IE창을 띄워서 'http://localhost:6006'을 주소창에 입력합니다.

학습 과정을 그래프로 볼 수 있는데, loss값들이 학습이 진행되면서 줄어드는 것을 알 수 있고, 제대로 학습이 되었다는 것을 알 수가 있습니다.

